### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Paths for files to load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()
school_data.head()



,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Compute number of schoos and students. Probably did not need the .unque after the students, but 
# added it just in case there is a student in more than one school, which is possible
no_schools=len(school_data_complete["School ID"].unique())
school_list=school_data_complete["School ID"].unique()
no_students=len(school_data_complete["Student ID"].unique())

In [3]:
# Compute total budget, used the original data file. Could have used combined dataframe, 
# but this was easier and the data frame was available.
total_budget=school_data["budget"].sum()

In [4]:
# Compute average math and reading scores
avg_math=school_data_complete["math_score"].mean()
avg_reading=school_data_complete["reading_score"].mean()

In [45]:
# Compute passing rates
# I would not have interpreted overall passing rate as the average of the scores, but 
# the percentage of students who passed both reading and math, but I am following directions here.

# Compute overall passing rate
pass_rate=(avg_math+avg_reading)/2

# Compute math passing rate
pass_math=school_data_complete["math_score"].ge(70)
no_pass_math=0
for i in pass_math:
    if i: no_pass_math += 1
pct_math=no_pass_math/no_students

# Compute reading passing rate
pass_reading=school_data_complete["reading_score"].ge(70)
no_pass_reading=0
for i in pass_reading:
    if i: no_pass_reading += 1
pct_reading=no_pass_reading/no_students

# For fun compute students passing both math and reading
no_pass_both=0
for index, row in school_data_complete.iterrows():
    if row["reading_score"]>=70 and row["math_score"]>=70:
        no_pass_both +=1
pct_both=no_pass_both/no_students


In [46]:
# Create a dataframe to hold above results
hold_df=pd.DataFrame({"Schools":[no_schools],
    "Students":[no_students],"Budget":[total_budget],
    "Average Math Score": [avg_math],"Average Reading Score":[avg_reading],
    "Overall Average Score":[pass_rate],"% Passing Math":[100*pct_math],
    "% Passing Reading":[100*pct_reading],"% Passing Both":[100*pct_both]})

# Optional formatting of dataframe holding results
hold_df_formatted=pd.DataFrame({"Schools":[no_schools],
    "Students":'{:,}'.format(no_students),"Budget":'$'+'{:,}'.format(total_budget),
    "Average Math Score": '{:6.2f}'.format(avg_math),"Average Reading Score":'{:6.2f}'.format(avg_reading),
    "Overall Avgerage Score":'{:6.2f}'.format(pass_rate),"% Passing Math":'{:.2%}'.format(pct_math),
    "% Passing Reading":'{:.2%}'.format(pct_reading),'% Passing Both':'{:.2%}'.format(pct_both)})

In [47]:
hold_df.head()

,Schools,Students,Budget,Average Math Score,Average Reading Score,Overall Average Score,% Passing Math,% Passing Reading,% Passing Both
0,15,39170,24649428,78.985371,81.87784,80.431606,74.980853,85.805463,65.172326


In [48]:
hold_df_formatted.head()

,Schools,Students,Budget,Average Math Score,Average Reading Score,Overall Avgerage Score,% Passing Math,% Passing Reading,% Passing Both
0,15,"39,170","$24,649,428",78.99,81.88,80.43,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [11]:
# start with school data file and append other information
result1_df=school_data
# Per Student Budget
add_2_df=school_data['budget']/school_data['size']

# Average Math Score from school_data
add_3_df=school_data_complete['math_score'].groupby(school_data_complete['school_name']).mean().to_frame('avg_math').reset_index()

# Average Reading Score from school_data
add_4_df=school_data_complete['reading_score'].groupby(school_data_complete['school_name']).mean().to_frame('avg_reading').reset_index()

# Portion with Math Score >= 70 from school_data_complete, converted to %
# [Note: This can be done this way because True = 1 and False = 0. Something I use often in coding.]
add_5a_df=100*school_data_complete['math_score'].ge(70).groupby(school_data_complete['school_name']).mean()
add_5_df=add_5a_df.to_frame('perc_math').reset_index()

# Portion with Reading Score >= 70 from school_data_complete, convertd to %
add_6a_df=100*school_data_complete['reading_score'].ge(70).groupby(school_data_complete['school_name']).mean()
add_6_df=add_6a_df.to_frame('perc_reading').reset_index()

# Overall Pass Rate
add_7a_df=(add_5a_df+add_6a_df)/2
add_7_df=add_7a_df.to_frame('perc_pass').reset_index()


# Create dataframe with results (and remove School ID column)
result1_df['bud_per_stud']=add_2_df
result1_df = pd.merge(result1_df, add_3_df, how="left", on=["school_name", "school_name"])
result1_df = pd.merge(result1_df, add_4_df, how="left", on=["school_name", "school_name"])
result1_df = pd.merge(result1_df, add_5_df, how="left", on=["school_name", "school_name"])
result1_df = pd.merge(result1_df, add_6_df, how="left", on=["school_name", "school_name"])
result1_df = pd.merge(result1_df, add_7_df, how="left", on=["school_name", "school_name"])
result1_df=result1_df.drop(columns='School ID')


result1_df

# This is not cleanly formatted, but could be done, as it was above.

,school_name,type,size,budget,bud_per_stud,avg_math,avg_reading,perc_math,perc_reading,perc_pass
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [12]:
# sort in descending order of percent passing and output the head
top_df=result1_df.sort_values(by=['perc_pass'],ascending=False)
top_df.head()

,school_name,type,size,budget,bud_per_stud,avg_math,avg_reading,perc_math,perc_reading,perc_pass
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [13]:
# sort in ascending order of percent passing and output the head
bot_df=result1_df.sort_values(by=['perc_pass'])
bot_df.head()

,school_name,type,size,budget,bud_per_stud,avg_math,avg_reading,perc_math,perc_reading,perc_pass
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [14]:
# I think there is a typo in the instructions. They read "average Reading Score", but I think it
# should read "average Math Score" based on title and next section, so I used math scores.
# I do not understand why we form a SERIES, then group the series, and then create a dataframe.
# A multi-groupby of the dataframe could get the same results much easier. (Provided later.) 

# initiate series 
val9=[]
val10=[]
val11=[]
val12=[]
index9=[]
index10=[]
index11=[]
index12=[]

#loop through each row 
for index, row in school_data_complete.iterrows():

# Add extracted information to the appropriate arrays
    id=row['School ID']
    score=row['math_score']
    if row["grade"]=="9th":
        val9.append(score)
        index9.append(id)
    elif row["grade"]=="10th":
        val10.append(score)
        index10.append(id)
    elif row["grade"]=="11th":
        val11.append(score)
        index11.append(id)
    elif row["grade"]=="12th":
        val12.append(score)
        index12.append(id)
    
# Curse a panda if it does not work    ;-)
    else:
        print("Die, Ling-Ling, Die!")
        



In [15]:
# Convert arrays to series with indices = school_id

ser9 = pd.Series(val9,index=index9)
ser10 = pd.Series(val10,index=index10)
ser11 = pd.Series(val11,index=index11)
ser12 = pd.Series(val12,index=index12)

# Convert series into nested dictionaries and then to a dataframe with formatting
flag=0
for sid in school_list:
    sc_nm = school_data.iloc[int(sid),1]
    mean9='{:7.3}'.format(ser9[sid].mean())
    mean10='{:7.3}'.format(ser10[sid].mean())
    mean11='{:7.3}'.format(ser11[sid].mean())
    mean12='{:7.3}'.format(ser12[sid].mean())
     
    if flag==0:
        dict={sc_nm: {'9th': mean9, '10th': mean10, '11th': mean11, '12th': mean12}}
        flag=1
    else:
        dict[sc_nm]={'9th': mean9, '10th': mean10, '11th': mean11, '12th': mean12}


In [16]:
math_frame=pd.DataFrame(dict)
# Transpose data frame to make it more readable.
math_frame.T

,9th,10th,11th,12th
Huang High School,77.0,75.9,76.4,77.2
Figueroa High School,76.4,76.5,76.9,77.2
Shelton High School,83.4,82.9,83.4,83.8
Hernandez High School,77.4,77.3,77.1,77.2
Griffin High School,82.0,84.2,83.8,83.4
Wilson High School,83.1,83.7,83.2,83.0
Cabrera High School,83.1,83.2,82.8,83.3
Bailey High School,77.1,77.0,77.5,76.5
Holden High School,83.8,83.4,85.0,82.9
Pena High School,83.6,83.4,84.3,84.1


In [17]:
# Here is an easy wat to get the results, though I did not format them
# This is for illustrative purposes only

op1_df=school_data_complete.groupby(['grade','school_name']).mean()
op1_df.drop(columns=["Student ID","School ID","size","budget"])


reading_score  math_score
grade school_name                                     
10th  Bailey High School         80.907183   76.996772
      Cabrera High School        84.253219   83.154506
      Figueroa High School       81.408912   76.539974
      Ford High School           81.262712   77.672316
      Griffin High School        83.706897   84.229064
      Hernandez High School      80.660147   77.337408
      Holden High School         83.324561   83.429825
      Huang High School          81.512386   75.908735
      Johnson High School        80.773431   76.691117
      Pena High School           83.612000   83.372000
      Rodriguez High School      80.629808   76.612500
      Shelton High School        83.441964   82.917411
      Thomas High School         84.254157   83.087886
      Wilson High School         84.021452   83.724422
      Wright High School         83.812757   84.010288
11th  Bailey High School         80.945643   77.515588
      Cabrera High School        83.788382   82.765560
      Figueroa High School       80.640339   76.884344
      Ford High School           80.403642   76.918058
      Griffin High School        84.288089   83.842105
      Hernandez High School      81.396140   77.136029
      Holden High School         83.815534   85.000000
      Huang High School          81.417476   76.446602
      Johnson High School        80.616027   77.491653
      Pena High School           84.335938   84.328125
      Rodriguez High School      80.864811   76.395626
      Shelton High School        84.373786   83.383495
      Thomas High School         83.585542   83.498795
      Wilson High School         83.764608   83.195326
      Wright High School         84.156322   83.836782
12th  Bailey High School         80.912451   76.492218
      Cabrera High School        84.287958   83.277487
      Figueroa High School       81.384863   77.151369
      Ford High School           80.662338   76.179963
      Griffin High School        84.013699   83.356164
      Hernandez High School      80.857143   77.186567
      Holden High School         84.698795   82.855422
      Huang High School          80.305983   77.225641
      Johnson High School        81.227564   76.863248
      Pena High School           84.591160   84.121547
      Rodriguez High School      80.376426   77.690748
      Shelton High School        82.781671   83.778976
      Thomas High School         83.831361   83.497041
      Wilson High School         84.317673   83.035794
      Wright High School         84.073171   83.644986
9th   Bailey High School         81.303155   77.083676
      Cabrera High School        83.676136   83.094697
      Figueroa High School       81.198598   76.403037
      Ford High School           80.632653   77.361345
      Griffin High School        83.369193   82.044010
      Hernandez High School      80.866860   77.438495
      Holden High School         83.677165   83.787402
      Huang High School          81.290284   77.027251
      Johnson High School        81.260714   77.187857
      Pena High School           83.807273   83.625455
      Rodriguez High School      80.993127   76.859966
      Shelton High School        84.122642   83.420755
      Thomas High School         83.728850   83.590022
      Wilson High School         83.939778   83.085578
      Wright High School         83.833333   83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [18]:
# initiate series, do not need school indices, since they are the same as for math
rval9=[]
rval10=[]
rval11=[]
rval12=[]

#loop through each row 
for index, row in school_data_complete.iterrows():

# Add extracted information to the appropriate arrays
    score=row['reading_score']
    if row["grade"]=="9th":
        rval9.append(score)
    elif row["grade"]=="10th":
        rval10.append(score)
    elif row["grade"]=="11th":
        rval11.append(score)
    elif row["grade"]=="12th":
        rval12.append(score)

    
# Curse a panda if it does not work ...
    else:
        print("Die, Hsing-Hsing, Die!")

# Convert arrays to series with indices = school_id

rser9 = pd.Series(rval9,index=index9)
rser10 = pd.Series(rval10,index=index10)
rser11 = pd.Series(rval11,index=index11)
rser12 = pd.Series(rval12,index=index12)

# Convert series into nested dictionaries and then to a dataframe with formatting
flag=0
for sid in school_list:
    sc_nm = school_data.iloc[int(sid),1]
    mean9='{:7.3}'.format(rser9[sid].mean())
    mean10='{:7.3}'.format(rser10[sid].mean())
    mean11='{:7.3}'.format(rser11[sid].mean())
    mean12='{:7.3}'.format(rser12[sid].mean())
     
    if flag==0:
        rdict={sc_nm: {'9th': mean9, '10th': mean10, '11th': mean11, '12th': mean12}}
        flag=1
    else:
        rdict[sc_nm]={'9th': mean9, '10th': mean10, '11th': mean11, '12th': mean12}  


In [19]:
reading_frame=pd.DataFrame(rdict)
reading_frame.T

,9th,10th,11th,12th
Huang High School,81.3,81.5,81.4,80.3
Figueroa High School,81.2,81.4,80.6,81.4
Shelton High School,84.1,83.4,84.4,82.8
Hernandez High School,80.9,80.7,81.4,80.9
Griffin High School,83.4,83.7,84.3,84.0
Wilson High School,83.9,84.0,83.8,84.3
Cabrera High School,83.7,84.3,83.8,84.3
Bailey High School,81.3,80.9,80.9,80.9
Holden High School,83.7,83.3,83.8,84.7
Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [20]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [24]:
# cut into bins and add the assigned group name as another column
result1_df['spend_bin']=pd.cut(result1_df['bud_per_stud'],spending_bins,labels=group_names)
# group by bin and produce the average 
group_spend=result1_df.groupby(result1_df['spend_bin'])
spend_df=group_spend.mean()
# This produces an average of the schools, which may be different than the average of the students 
# in those schools. 
spend_df.drop(columns=['size','budget','bud_per_stud'])

,avg_math,avg_reading,perc_math,perc_reading,perc_pass
spend_bin,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [28]:
# cut into bins and add the assigned group name as another column
result1_df['size_group']=pd.cut(result1_df['size'],size_bins,labels=group_size_names)
# group by bin and produce the average  
group_size=result1_df.groupby(result1_df['size_group'])
size_df=group_size.mean()
# This produces an average of the schools, which may be different than the average of the students 
# in those schools. 
size_df.drop(columns=['size','budget','bud_per_stud'])

,avg_math,avg_reading,perc_math,perc_reading,perc_pass
size_group,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [29]:
# Since the type is already in the dataframe, group by type.

group_type=result1_df.groupby(result1_df['type'])
type_df=group_type.mean()
type_df.drop(columns=['size','budget','bud_per_stud'])

,avg_math,avg_reading,perc_math,perc_reading,perc_pass
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
# Note: The analyses measure the performance of schools and should not be applied to students.
# A different set of metrics would need to be used to determine the performance of students.
# A weighted average of student scores would be used if we wanted to know the statistical 
# expectations of a student in a particular bin. 

# Some interesting "conclusions" could be drawn from these results, such as:
# The less we spend per student the better they tend to perform. Or ...
# Charter schools are superior to district schools.
# The relationship between scores and spending per student or type of school
# could be impacted by other factors. Additional spending often increases
# bureaucratic demands on the teachers, taking them away from teaching. In the 1980s a study of
# this type was performed on public schools in Texas and there was an inverse relationship between
# the inflation adjusted amount spent per student and the overall SAT scores. Further investigation
# revealed that additional funding did not support teachers or result in them being paid more, 
# but added bureacratic layers which increased the non-teaching demands on teachers and hurt the
# retention rate of good teachers. On a more granular level, additional spending may also be
# associated with additional costs in helping students from less educated and lower
# income households. These students tend to score lower in testing due to factors not provided
# in this data. Similarly the socio-economic and educational backgrounds of the students is not
# considered in the charter vs district school. The results of this analysis only point to areas to 
# further investigate and should not be conclusive based solely on the data provided. 
# Norming of results would be appropriate to better assess the actual impact of spending, charter
# schools, school size, etc. 

# P.S. I am not a liberal. Facts are often confused for truth. Truth is superior to facts.
# Although there can be no actual conflict between facts and truth, to understand facts may require
# that they are viewed within the realm of a greater truth. This is an excellent example of how 
# facts have the potential of being used to distort the truth (i.e. ethical considerations.)